## Automated LinkedIn Learning Course Extractor

In [ ]:
##libraries
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
from pandas import ExcelWriter 

In [ ]:
print("\033[1m"+"Automated LinkedIn Learning Course Extractor"+"\033[0m")

field=input("In which field you want to search courses:")

courses_title_and_link_table=pd.DataFrame(columns=["Course_title","Duration","instructor","Link_to_course"])
courses_title_and_link_table


driver = Chrome(executable_path='/home/rahuldevs/chromedriver_linux64/chromedriver')

driver.get("https://www.linkedin.com/login")



email_field = driver.find_element(By.ID,"username")
password_field = driver.find_element(By.ID,"password")
email_field.send_keys("anshuu5555@gmail.com")
password_field.send_keys("rahul@123456")
password_field.submit()

element= WebDriverWait(driver, 30).until(
    EC.visibility_of_element_located((By.ID, "global-nav-search")))

print("Initiating Programe...")
search=driver.find_element(By.XPATH,"/html/body/div[5]/header/div/div/div/button")
search_input=driver.find_element(By.XPATH,"/html/body/div[5]/header/div/div/div/div[1]/input")
search.click()
search_input.send_keys(field)
search_input.send_keys(Keys.ENTER)

driver.implicitly_wait(10)
courses_option=driver.find_elements(By.CLASS_NAME,"artdeco-pill.artdeco-pill--slate.artdeco-pill--choice.artdeco-pill--2.search-reusables__filter-pill-button")
for option in courses_option:
    if option.text=="Courses":
        courses_option=option
        break
courses_option.click()
element= WebDriverWait(driver, 30).until(
    EC.visibility_of_element_located((By.XPATH, "/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[4]/div/div/button[2]")))
print("successfully initiated ")
print("Extraction begin...")
def courses(button_xpath):
    courses_list = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/ul")
    courses = courses_list.find_elements(By.XPATH, "./li")
    for course in courses:
        course_name = course.find_element(By.TAG_NAME, "strong").text
        duration=course.find_element(By.CLASS_NAME,"entity-result__primary-subtitle.t-14.t-black.t-normal").text
        duration=duration.split("•")[1].strip()
        
        instructor=course.find_element(By.CLASS_NAME,"entity-result__secondary-subtitle.t-14.t-normal").text
        
        instructor = instructor.split(":")[1].split("•")[0].strip()
        
        course_link = course.find_element(By.TAG_NAME, "a").get_attribute("href")
        courses_title_and_link_table.loc[len(courses_title_and_link_table)]=[course_name,duration,instructor,course_link]
    button=driver.find_element(By.XPATH,button_xpath)
    return button
    

courses("/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[4]/div/div/button[2]").click()

while True:
    driver.implicitly_wait(20) 

    button=courses("/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[2]/div/div[2]/div/button[2]")
    if button.is_enabled():
        button.click()
        continue
    else:
        break
print("successfully Extracting!")  


print(f"Saving into {field}.xlsx file...")


driver.quit()
courses_title_and_link_table.to_excel(f"{field}_courses.xlsx")
with ExcelWriter(f"{field}_courses.xlsx") as writer:
    courses_title_and_link_table.to_excel(writer, sheet_name='Sheet1', index=False)
    worksheet = writer.sheets['Sheet1']
    worksheet.set_column('A:Z', 20)

print("Finished!")